In [15]:
import pandas as pd
import re

In [7]:
data=pd.read_csv("train.csv", encoding='ISO-8859-1',names=["target", "ids", "date", "flag", "user", "text"])

In [10]:
data.text[0]

"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"

In [11]:
x=data["text"]

In [12]:
y=data["target"]

In [27]:
new_text=[]
for i in x:
    text=i
    text.lower()
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text,  
                  flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    new_text.append(text)

In [30]:
clean_text=[]
from nltk.corpus import stopwords
for i in new_text:
    word=i.split()
    stops = set(stopwords.words("english"))
    word=[w for w in word if not w in stops]
    word = " ".join(word)
    clean_text.append(word)
    
    
    

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [34]:
vect.fit(clean_text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [35]:
x=vect.transform(clean_text)

In [36]:
vect.get_feature_names()

['00',
 '000',
 '0000',
 '00000',
 '000000000000',
 '0000001',
 '000001',
 '000014',
 '00009',
 '0000ff',
 '0001110101001010000111',
 '0001t',
 '0007',
 '000aah',
 '000albums',
 '000followers',
 '000follows',
 '000ft',
 '000g',
 '000gbp',
 '000h',
 '000hrs',
 '000ish',
 '000k',
 '000kg',
 '000km',
 '000kms',
 '000mah',
 '000martha',
 '000mnt',
 '000pv',
 '000s',
 '000th',
 '000thb',
 '000watts',
 '000webhost',
 '000yrs',
 '001',
 '0010utc',
 '0010x0010',
 '0011000100101011001100010011110100110010',
 '0012',
 '0014',
 '001st',
 '002',
 '00202',
 '003',
 '003r',
 '003ï',
 '004',
 '0047',
 '005',
 '0055',
 '005587',
 '0055gb',
 '005mm',
 '006',
 '0069',
 '007',
 '0074',
 '007episode',
 '007heather007',
 '007hil',
 '007jbond',
 '007louiseob',
 '007peter',
 '007wifey',
 '007wisdom',
 '008',
 '0084',
 '00a',
 '00am',
 '00b1',
 '00ff00',
 '00fieldsy',
 '00g',
 '00ga',
 '00geneva',
 '00h',
 '00hrs',
 '00hs',
 '00jon',
 '00k',
 '00mathias',
 '00mony00',
 '00neji',
 '00o',
 '00oo',
 '00p',
 '00p

In [39]:
x.toarray

<bound method _cs_matrix.toarray of <1600000x596692 sparse matrix of type '<class 'numpy.int64'>'
	with 12135772 stored elements in Compressed Sparse Row format>>

In [40]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [41]:
y=le.fit_transform(y)

In [42]:

from keras.models import Sequential
from keras.layers import Dense,Flatten,Conv2D,MaxPool2D,Dropout
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping 

Using TensorFlow backend.


In [45]:
from sklearn.model_selection import train_test_split

In [46]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42,stratify=y)

In [47]:
from sklearn.naive_bayes import MultinomialNB

In [48]:
nb=MultinomialNB()

In [49]:
nb.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [50]:
y_pred=nb.predict(x_test)
y_pred.shape

(480000,)

In [51]:
count=0
for i in range(len(y_pred)):
    if y_pred[i]==y_test[i]:
        count=count+1

In [52]:
print(count)

369099


In [53]:
print("accuracy:",count/len(y_pred))

accuracy: 0.76895625


In [54]:
set(y)

{0, 1}

In [58]:
y=len(vect.get_feature_names())
y

596692

In [77]:
model = Sequential()

#num_words = len(word_index) + 1

#input_shape = train_feature_cv.shape[1]
model.add(Dense(50,activation='relu', input_shape=(596692,)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 50)                29834650  
_________________________________________________________________
dropout_4 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 51        
Total params: 29,834,701
Trainable params: 29,834,701
Non-trainable params: 0
_________________________________________________________________
None


In [79]:
model.fit(x_train,y_train, epochs=10, batch_size=1000)

ValueError: Input arrays should have the same number of samples as target arrays. Found 1120000 input samples and 596693 target samples.

In [73]:
y_train = to_categorical(y)

In [65]:
y_train.shape

(596693,)

AttributeError: 'int' object has no attribute 'shape'